<a href="https://colab.research.google.com/github/Amine-kouki/Intent-Classifier-AI-Agent/blob/main/Data_cleanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
%pip install pandas datasets transformers scikit-learn

### Loading and extracting thr JSON date

In [ ]:
import json
import os

try:
    with open('intents_dataset.json', 'r') as f:
        data = json.load(f)
        print("Data loaded successfully.")
        print(data)  # Print first two intents for inspection

except FileNotFoundError:
    print("Error: The file was not found.")
except json.JSONDecodeError:
    print("Error: Failed to decode JSON from the file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



Data loaded successfully.
{'software_issue': {'cannot_login': ["I can't log in", "My password isn't working", "I'm locked out of my account", 'It says my credentials are invalid', 'I forgot my password', "My login isn't being accepted", "The system won't let me in", "I keep getting an 'access denied' message", 'I need a password reset', 'It says my account is disabled', "I'm stuck on the login screen", "My two-factor authentication isn't working", "I get a 'wrong username or password' error", "I changed my password and now it doesn't work", 'The login page just keeps refreshing'], 'application_crash': ['The program keeps crashing', 'My software keeps closing unexpectedly', 'I keep getting an error message and then it closes', 'It freezes and I have to force quit', 'The application is not responding', "It just shuts down randomly while I'm working", 'I get a pop-up that says [Application] has stopped working', 'My screen goes grey and I have to restart the program', 'It hangs every time

In [ ]:
import pandas as pd

rows = []
for category, intents in data.items():
    for intent , messages in intents.items():
        for messages in messages :
            rows.append({'category': category, 'intent': intent, 'text': messages})

df = pd.DataFrame(rows)

print(df.head())

         category        intent                                text
0  software_issue  cannot_login                      I can't log in
1  software_issue  cannot_login           My password isn't working
2  software_issue  cannot_login        I'm locked out of my account
3  software_issue  cannot_login  It says my credentials are invalid
4  software_issue  cannot_login                I forgot my password


#### Basic normalization

In [ ]:
print(df.columns)

Index(['category', 'intent', 'text'], dtype='object')


In [ ]:
import re
import pandas as pd

def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = ' '.join(text.split()).strip()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'[^\w\s\.\,\!\?\'\d\-\/]+', ' ', text)
        text = ' '.join(text.split()).strip()
    return text

df['cleaned_text'] = df['text'].apply(normalize_text)

df.to_csv('cleaned_intents_dataframe.csv', index=False)

print("DataFrame head (showing original vs. cleaned text):")
print(df[['text', 'cleaned_text', 'intent']].head())
print("\nDataFrame info:")
df.info()

DataFrame head (showing original vs. cleaned text):
                                 text                        cleaned_text  \
0                      I can't log in                      i can't log in   
1           My password isn't working           my password isn't working   
2        I'm locked out of my account        i'm locked out of my account   
3  It says my credentials are invalid  it says my credentials are invalid   
4                I forgot my password                i forgot my password   

         intent  
0  cannot_login  
1  cannot_login  
2  cannot_login  
3  cannot_login  
4  cannot_login  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      446 non-null    object
 1   intent        446 non-null    object
 2   text          446 non-null    object
 3   cleaned_text  446 non-null    object
dtyp

### Tokenization

In [ ]:
# Install transformers in this Jupyter environment if it's not already installe

from transformers import AutoTokenizer

Model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(Model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
text_data = df['cleaned_text'].tolist()

tokenized_data = tokenizer(
    text_data,
    padding=True ,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

### Load the Pre-trained Model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels=num_labels)